<h1>Binarization</h1>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
import scipy.stats as stats  #for QQ plot
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer,KBinsDiscretizer




In [26]:
df = pd.read_csv('train.csv' , usecols=['Age' , 'Fare' , 'Survived' , 'SibSp' , 'Parch'])

In [27]:
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [28]:
df.dropna(inplace=True)

In [29]:
df.isnull().sum()

Survived    0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64

In [30]:
df['Family'] = df['SibSp'] + df['Parch']

In [31]:
df.head()

,Survived,Age,SibSp,Parch,Fare,Family
0,0,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,1
2,1,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,1
4,0,35.0,0,0,8.0500,0


In [32]:
df.drop(columns=['SibSp' , 'Parch'] , inplace=True)

In [35]:
df.head()

,Survived,Age,Fare,Family
0,0,22.0,7.2500,1
1,1,38.0,71.2833,1
2,1,26.0,7.9250,0
3,1,35.0,53.1000,1
4,0,35.0,8.0500,0


In [36]:
X = df.drop(columns=['Survived'])
Y = df['Survived']

In [37]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, random_state=50)

In [38]:
X_train.head()

,Age,Fare,Family
153,40.5,14.5000,2
104,37.0,7.9250,2
606,30.0,7.8958,0
416,34.0,32.5000,2
678,43.0,46.9000,7


In [39]:
clf = DecisionTreeClassifier()
clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)
accuracy_score(Y_test,Y_pred)

0.6223776223776224

In [40]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,Y,cv=10,scoring='accuracy'))

np.float64(0.6471048513302033)

In [41]:
# Applying Binarizer
from sklearn.preprocessing import Binarizer


In [46]:
trf = ColumnTransformer([
    ('bin' , Binarizer(copy=False) , ['Family'])
],remainder='passthrough')

In [47]:
X_train_trfd = trf.fit_transform(X_train)
X_test_trfd = trf.fit_transform(X_test)

In [48]:
pd.DataFrame(X_train_trfd,columns=['Family' , 'Age' , 'Fare'])

,Family,Age,Fare
0,1.0,40.5,14.5000
1,1.0,37.0,7.9250
2,0.0,30.0,7.8958
3,1.0,34.0,32.5000
4,1.0,43.0,46.9000
...,...,...,...
566,1.0,1.0,39.6875
567,1.0,29.0,27.7208
568,1.0,37.0,53.1000
569,1.0,22.0,41.5792


In [49]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trfd , Y_train)
y_pred = clf.predict(X_test_trfd)
accuracy_score(Y_test,y_pred)

0.5874125874125874

In [50]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier() , X_trf , Y , cv=10 , scoring='accuracy'))

np.float64(0.6261932707355242)